In [ ]:
import monologue
#we use this to get fenced stuff out
from monologue.core.utils.ops import parse_fenced_code_blocks
#saves typing, fairly basic- generate pydantic types from example data
from monologue.core.utils.ops import pydantic_type_generator
#the entities we will create extend this which does the repr and pulls out some metadata
from monologue.entities import AbstractEntity


## Illustrate the type representation in logs

In [ ]:
from pydantic import Field
class MyEntity(AbstractEntity):
    code: str = Field(is_key=True)
    created_at: str

my_entity = MyEntity(code='test', created_at= "2023-01-01")
my_entity

## Show how the Columnar Store works

In [ ]:
from monologue.core.stores import ColumnarDataStore
from monologue.entities.examples import NycTripEvent
store = ColumnarDataStore(NycTripEvent)
store

### how we add data to the store

In [ ]:
import pandas as pd
data = pd.read_csv("/Users/sirsh/Downloads/nyc_trip_data_sample.csv") 
store.add(data)

### load the tool and ask questions

In [ ]:
tool = store.as_tool()
tool

In [ ]:
tool.run("What is least popular destination in new york city? Who has travelled there?")

# Vector Store Loading

In [1]:
import pandas as pd
from monologue.core.stores import VectorDataStore
data = pd.read_csv("/Users/sirsh/Downloads/marvel_bios.csv").rename(columns={'entity_key':'id'})
data.head()

,entity_type,id,text
0,people,"Henry Jonathan ""Hank"" Pym",\n\nGiant-Man\n\n\n\n\n\nGallery\n\nName\nDr. ...
1,people,Janet van Dyne,NaN
2,people,"Anthony Edward ""Tony"" Stark",\n\nIron Man\n\n\n\n\n\nGallery\n\nName\nAntho...
3,people,Robert Bruce Banner,NaN
4,people,Thor Odinson,\n\nThor\n\n\n\n\n\nGallery\n\nName\nThor Odin...


In [2]:
from monologue.entities.examples import *
store = VectorDataStore(AvengingPassengers)

In [3]:
#store.add(data)
df = store.load()
df

,entity_type,id,text,vector
0,people,"Henry Jonathan ""Hank"" Pym",\n\nGiant-Man\n\n\n\n\n\nGallery\n\nName\nDr. ...,"[-0.016914167, -0.021530049, 0.01588479, -0.02..."
1,people,"Anthony Edward ""Tony"" Stark",\n\nIron Man\n\n\n\n\n\nGallery\n\nName\nAntho...,"[-0.019250032, -0.032906756, -0.0071217827, -0..."
2,people,Thor Odinson,\n\nThor\n\n\n\n\n\nGallery\n\nName\nThor Odin...,"[-0.012796414, -0.022400342, -0.014959867, -0...."
3,people,Richard Milhouse Jones,\n\nRick Jones\n\n\n\n\n\nGallery\n\nName\nRic...,"[-0.04045754, -0.029762171, 0.012101611, -0.01..."
4,people,Steven Rogers,\n\nCaptain America\n\n\n\n\n\nGallery\n\nName...,"[-0.016223148, -0.045067456, -0.0012523064, -0..."
...,...,...,...,...
158,people,Victor Alvarez,\n\nPower Man\n\n\n\n\n\nGallery\n\nName\nVíct...,"[-0.0036144522, -0.036405325, 0.0077103856, -0..."
159,people,Ava Ayala,\n\nWhite Tiger\n\n\n\n\n\nGallery\n\nName\nAv...,"[-0.019321058, -0.035417497, -0.031073593, -0...."
160,people,Kaluu,\n\nKaluu\n\n\n\n\n\nGallery\n\nName\nKaluu\n\...,"[0.013132852, -0.018956985, 0.027421124, -0.02..."
161,None,-1,test,"[-0.0017278135, -0.013941668, 0.0016401962, -0..."


In [7]:
tool = store.as_tool(debug_db=False,model='gpt-3.5-turbo')
tool

Tool(name='Further details tool relate to AvengingPassengers entities', description='If and only if the other tools return no results, use this tool to get extra information about any AvengingPassengers entity that you are asked about.  \n                Do not pass identifiers and codes to this tool. Only pass proper nouns and questions in full sentences.\n                Added context: \n                About the entity: None\n                ', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verb

In [8]:
tool.run("What can you tell me about Steven Rogers?")

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 47250 tokens. Please reduce the length of the messages.

In [12]:
data = pd.read_csv("/Users/sirsh/Downloads/nyc_zones.csv").drop(columns='id',index=1).rename(columns={'entity_key':'id'})
data.head()

,index,entity_type,id,text
0,0,nyc_zone,Midtown Center,Page: Midtown Manhattan\nSummary: Midtown Manh...
2,2,nyc_zone,Clinton East,Page: Bill Clinton\nSummary: William Jefferson...
3,3,nyc_zone,LaGuardia Airport,Page: LaGuardia Airport\nSummary: LaGuardia Ai...
4,4,nyc_zone,Gramercy,Page: Gramercy Park\nSummary: Gramercy Park ()...
5,5,nyc_zone,Upper West Side South,Page: Upper West Side\nSummary: The Upper West...


In [13]:
store = VectorDataStore(Places)
store.add(data)

2023-10-01 17:39:47.170 | INFO     | monologue.core.stores.VectorStore:add:53 - Adding 101 to s3://res-data-platform/stores/vector/v0/examples_Places...


,index,entity_type,id,text
0,0,nyc_zone,Midtown Center,Page: Midtown Manhattan\nSummary: Midtown Manh...
2,2,nyc_zone,Clinton East,Page: Bill Clinton\nSummary: William Jefferson...
3,3,nyc_zone,LaGuardia Airport,Page: LaGuardia Airport\nSummary: LaGuardia Ai...
4,4,nyc_zone,Gramercy,Page: Gramercy Park\nSummary: Gramercy Park ()...
5,5,nyc_zone,Upper West Side South,Page: Upper West Side\nSummary: The Upper West...
...,...,...,...,...
97,97,nyc_zone,Bronxdale,"Page: Allerton, Bronx\nSummary: Allerton is a ..."
98,98,nyc_zone,East New York,"Page: East New York, Brooklyn\nSummary: East N..."
99,99,nyc_zone,Saint Michaels Cemetery/Woodside,"Page: Woodside, Queens\nSummary: Woodside is a..."
100,100,nyc_zone,Soundview/Castle Hill,"Page: Castle Hill, Bronx\nSummary: Castle Hill..."


In [ ]:
tool = store.as_tool(debug_db=False)
tool.run("What can you tell me about Gramercy?")